In [ ]:
#
# Решение
#
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
!wget https://storage.yandexcloud.net/academy.ai/cat-and-dog.zip

# Разархивируем датасет во временную папку 'temp'
!unzip -qo "cat-and-dog" -d ./temp

# Папка с папками картинок, рассортированных по категориям
IMAGE_PATH = './temp/training_set/training_set/'

# Папка в которой будем создавать выборки
BASE_DIR = './dataset/'

# Создаем папки для обучающей и валидационной выборок
os.makedirs(BASE_DIR, exist_ok=True)

# Создаем генераторы данных с аугментацией
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% данных будет использовано для валидации
)

# Генератор для обучающей выборки
train_generator = datagen.flow_from_directory(
    IMAGE_PATH,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Генератор для валидационной выборки
validation_generator = datagen.flow_from_directory(
    IMAGE_PATH,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Загружаем предобученную модель MobileNet
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Добавляем GlobalAveragePooling2D и Dense слои
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

# Создаем модель
model = Model(inputs=base_model.input, outputs=predictions)

# Замораживаем слои базовой модели
for layer in base_model.layers:
    layer.trainable = False

# Компилируем модель
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Обучаем модель
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

# Оцениваем модель на валидационной выборке
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

# Если точность низкая, то попробуем разморозить некоторые слои базовой модели
# и продолжитьс более низкой скоростью обучения
for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

# Оцениваем модель на валидационной выборке после тонкой настройки
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy after fine-tuning: {accuracy * 100:.2f}%')

--2024-10-17 13:07:16--  https://storage.yandexcloud.net/academy.ai/cat-and-dog.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228082266 (218M) [application/x-zip-compressed]
Saving to: ‘cat-and-dog.zip’

cat-and-dog.zip     100%[===================>] 217.52M  16.2MB/s    in 13s     

2024-10-17 13:07:31 (16.3 MB/s) - ‘cat-and-dog.zip’ saved [228082266/228082266]

Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.
17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


200/200 ━━━━━━━━━━━━━━━━━━━━ 108s 485ms/step - accuracy: 0.9236 - loss: 0.2140 - val_accuracy: 0.9762 - val_loss: 0.0605
Epoch 2/10
  1/200 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.8750 - loss: 0.1737

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8750 - loss: 0.1737 - val_accuracy: 1.0000 - val_loss: 1.3828e-05
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 94s 459ms/step - accuracy: 0.9693 - loss: 0.0821 - val_accuracy: 0.9538 - val_loss: 0.1258
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9688 - loss: 0.1677 - val_accuracy: 1.0000 - val_loss: 3.4660e-04
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 93s 456ms/step - accuracy: 0.9808 - loss: 0.0564 - val_accuracy: 0.9663 - val_loss: 0.0764
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 1.0000 - loss: 0.0140 - val_accuracy: 1.0000 - val_loss: 1.3732e-04
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 141s 459ms/step - accuracy: 0.9795 - loss: 0.0505 - val_accuracy: 0.9769 - val_loss: 0.0586
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 1.0000 - loss: 0.0279 - val_accuracy: 1.0000 - val_loss: 4.0618e-04
Epoch 9/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 93s 456ms/step - accuracy: 0.9758 - loss: 0.0